In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from skimpy import skim

In [3]:
from statsforecast.models import auto_arima
from statsforecast.core import StatsForecast
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import  BottomUp, TopDown, MiddleOut, MinTrace

In [4]:
from sklearn.metrics import mean_squared_error as mse

In [5]:
def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

In [6]:
FIG_SIZE = (20, 10)
plt.rcParams.update({'figure.figsize': FIG_SIZE})   

In [7]:
calender_df = pd.read_csv('data/calendar.csv', parse_dates=['date'], usecols=['date', 'wm_yr_wk', 'd'])

In [8]:
df = pd.read_csv('data/sales_train_evaluation.csv').loc[lambda df_: df_['item_id'] == 'FOODS_3_819']

In [9]:
df = (df
     .melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
     .drop(columns=['id'])
     .merge(calender_df, left_on='variable', right_on='d', how='left')
     .rename(columns={'value': 'sales_qty'})
     .loc[lambda df_: df_.date >= "2014-01-01", ['date', 'store_id', 'sales_qty']]
     .assign(state_id=lambda df_: df_['store_id'].str[:2])
     # .drop(columns=['state_id'])
     .rename(columns={'date': 'ds', 'sales_qty': 'y', 'store_id': 'unique_id'})
     .sort_values(['ds'])
     .reset_index(drop=True)
    )

df.head()

,ds,unique_id,y,state_id
0,2014-01-01,CA_1,3,CA
1,2014-01-01,CA_2,0,CA
2,2014-01-01,CA_3,7,CA
3,2014-01-01,CA_4,0,CA
4,2014-01-01,TX_1,0,TX


In [10]:
df_tot = df.groupby('ds')[['y']].sum().assign(unique_id='total').reset_index()

In [11]:
df_state = df.groupby(['ds', 'state_id'])[['y']].sum().reset_index().rename(columns={'state_id': 'unique_id'})

In [12]:
df = pd.concat([df.drop(columns='state_id'), df_tot, df_state])

In [13]:
def count_dls(string, dl='_'):
    return string.count(dl)


count_dls = np.vectorize(count_dls, otypes=[np.int32], cache=True)


def make_summing_matrix(df, id_col='unique_id', dl='_'):
    rows = df['unique_id'].unique()
    dls = count_dls(rows)
    cols = rows[dls == dls.max()]

    matrix = (
              pd.DataFrame(np.zeros((len(rows), len(cols)), dtype=np.int32), index=rows, columns=cols)
              .sort_index()
              .apply(lambda s: pd.Series(s.index.str.contains(f'^{s.name}', regex=True).astype(np.int32), index=s.index), axis=1)
    )
    
    matrix.loc['total', :] = 1
    return matrix

In [14]:
summing_matrix = make_summing_matrix(df)
summing_matrix

,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
CA,1,1,1,1,0,0,0,0,0,0
CA_1,1,0,0,0,0,0,0,0,0,0
CA_2,0,1,0,0,0,0,0,0,0,0
CA_3,0,0,1,0,0,0,0,0,0,0
CA_4,0,0,0,1,0,0,0,0,0,0
TX,0,0,0,0,1,1,1,0,0,0
TX_1,0,0,0,0,1,0,0,0,0,0
TX_2,0,0,0,0,0,1,0,0,0,0
TX_3,0,0,0,0,0,0,1,0,0,0
WI,0,0,0,0,0,0,0,1,1,1


In [15]:
tags = {}
tags['Country'] = np.array(['total'], dtype=object)
tags['Country/State'] = np.array(['CA', 'TX', 'WI'], dtype=object)
tags['Country/State/Store'] = np.array(['CA_1', 'CA_2', 'CA_3', 'CA_4',  
                                        'TX_1', 'TX_2', 'TX_3',
                                        'WI_1', 'WI_2', 'WI_3'], dtype=object)

In [16]:
horizon = 7

X_test = df.groupby('unique_id').tail(horizon).copy()
X_train = df.drop(X_test.index).copy()

X_train.set_index('unique_id', inplace=True)
X_test.set_index('unique_id', inplace=True)

In [17]:
forecaster = StatsForecast(X_train, models=[(auto_arima, 7)], freq='D', n_jobs=-1)
forecasts = forecaster.forecast(horizon)

In [18]:
result = pd.merge(X_test, forecasts, on=['ds', 'unique_id'])
result.columns = [['ds', 'y_true', 'y_pred']]
print(f'Overall RMSE: {rmse(result["y_true"], result["y_pred"])}')
for tag in tags.keys():
    print(f'{tag} RMSE: {rmse(result.loc[tags[tag], "y_true"], result.loc[tags[tag], "y_pred"])}')

Overall RMSE: 2.3002841674471664
Country RMSE: 4.8226593735285865
Country/State RMSE: 2.9437948047795683
Country/State/Store RMSE: 1.5755149800331563


In [19]:
reconcilers = [BottomUp(), TopDown(method='forecast_proportions'), MiddleOut(level='Country/State', top_down_method='forecast_proportions'), MinTrace(method='ols')]
h_rec = HierarchicalReconciliation(reconcilers=reconcilers)
h_forecasts = h_rec.reconcile(forecasts, X_train, summing_matrix, tags)

evaluator = HierarchicalEvaluation(evaluators=[rmse])
evaluator.evaluate(h_forecasts, X_test, tags, benchmark='auto_arima_season_length-7')

,,auto_arima_season_length-7,auto_arima_season_length-7/BottomUp,auto_arima_season_length-7/TopDown_method-forecast_proportions,auto_arima_season_length-7/MiddleOut_level-Country/State_top_down_method-forecast_proportions,auto_arima_season_length-7/MinTrace_method-ols
level,metric,,,,,
Overall,rmse-scaled,1.0,0.981594,1.008006,1.061943,0.98397
Country,rmse-scaled,1.0,1.093323,1.0,1.122233,1.01841
Country/State,rmse-scaled,1.0,0.849297,0.961181,1.0,0.913524
Country/State/Store,rmse-scaled,1.0,1.0,1.061938,1.066821,1.021558
